In [1]:
import os
import pickle
import joblib

import pandas as pd
import numpy as np
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns
import shap

from pprint import pprint
from shutil import copyfile
from sklearn import metrics, dummy, linear_model, ensemble, svm, neural_network
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import GridSearchCV, train_test_split, LeaveOneOut

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.datasets import load_iris

In [2]:
config_path = os.path.abspath(r'C:\Users\USER\Guro_Psy_KJH Dropbox\1.Projects\1_anxiety_VR\3_Data\5_prediction_model\1_2023_report')
data = 'vrabes_preprocessed.pkl'

In [3]:
with open(data,'rb') as f:
    pre = pickle.load(f)

In [4]:
# define label
ses_01 = pd.concat([pre['ses-01_demo'], pre['ses-01_crf'], ], axis=1)
ses_02 = pd.concat([pre['ses-02_demo'], pre['ses-02_crf']], axis=1)

ses_01_anx = ses_01[ses_01['ses-01_group']==1] #31
ses_02_anx = ses_02[ses_02['ses-02_group']==1] #26

ses_01_anx['id'] = ses_01_anx.index
ses_02_anx['id'] = ses_02_anx.index

ses_01_anx['sub'] = ses_01_anx['id'].apply(lambda x: x.split('_')[0])
ses_02_anx['sub'] = ses_02_anx['id'].apply(lambda x: x.split('_')[0])

early_response = pd.merge(ses_02_anx, ses_01_anx, on = 'sub', how = 'left')
early_response['delta'] = (early_response['ses-01_PDSS_SUM'] - early_response['ses-02_PDSS_SUM_y'])/early_response['ses-01_PDSS_SUM']
early_response['label'] = early_response['delta']>0.25 # 12




A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [5]:
ses_01 = pd.concat([pre['ses-01_as'],pre['ses-01_demo'],pre['ses-01_hrv']], axis=1)
ses_01_anx = ses_01[ses_01['ses-01_group']==1] 
ses_01_anx['id'] = ses_01_anx.index
ses_01_anx['sub'] = ses_01_anx['id'].apply(lambda x: x.split('_')[0])
target = pd.merge(early_response[['sub','label']], ses_01_anx, on = 'sub', how='left')


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
def main(config_path,target):

    
    # prepare result path
    if not os.path.isdir(os.path.join(config_path,'reult_dir_3')):
        os.mkdir(os.path.join(config_path,'reult_dir_3'))
    os.mkdir(os.path.join(config_path,'reult_dir_3', 'fig'))
    os.mkdir(os.path.join(config_path,'reult_dir_3', 'fig', 'confusion_matrix'))
    os.mkdir(os.path.join(config_path,'reult_dir_3', 'fig', 'roc'))
    os.mkdir(os.path.join(config_path,'reult_dir_3', 'model'))
    os.mkdir(os.path.join(config_path,'reult_dir_3', 'result'))
    os.mkdir(os.path.join(config_path,'reult_dir_3', 'result', 'cv'))

    ### start analysis

    # define models
    models = [
        ('Dummy', dummy.DummyClassifier()),
        ('LogReg', linear_model.LogisticRegression()),
        ('SVM', svm.SVC()),
        ('RandomForest', ensemble.RandomForestClassifier()),
        ('MLP', neural_network.MLPClassifier())
    ]

    # define hyperparameter search space
    param_grid = {
        'Dummy': {
        
        },
        'LogReg': {
            'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga'],
            'penalty': ['l1', 'l2', 'elasticnet'],
            'C': [float(10**x) for x in range(-4, 4)],
            'l1_ratio': np.random.uniform(size=5),
            'max_iter': [10000]
        },
        'SVM': {
            'kernel': ['linear', 'rbf', 'poly'],
            'C': [float(10**x) for x in range(-3, 3)],
            'gamma': [float(10**x) for x in range(-3, 4)],
            'probability': [True],
        },
        'RandomForest': {
            'n_estimators': [10, 25, 100],            
            'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)]+[None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'bootstrap': [True, False]
        },
        'MLP': {
            'hidden_layer_sizes': [10, 50, 100],
            'solver': ['lbfgs', 'sgd', 'adam'],
            'alpha': [float(10**x) for x in range(-2,2)],
            'batch_size': [int(2**x) for x in range(3, 5)],
            'learning_rate': ['constant', 'invscaling', 'adaptive'],
            'max_iter': [1000],
            'warm_start': [False, True]
        },
        
    }

    # prepare input-label data pairs    
    X_pre = target.drop(['ses-01_group','label','sub','id'],axis =1).to_numpy()
    y = target['label'].to_numpy()

    # # feature selection
    feat_sel = SelectKBest(f_classif, k=5)
    X = feat_sel.fit_transform(X_pre,y)
    selected_feature = {0:feat_sel.get_feature_names_out()}    
    pd.DataFrame.from_dict(selected_feature).to_csv(os.path.join(config_path,'reult_dir_3', 'result', 'selected_features.csv'))
    

    # split data pairs into train and test datasets
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=123, shuffle=True, stratify=y)

    # standardize input data
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    # LOOCV 
    loo = LeaveOneOut()

    # start model training
    result_dict = {}
    weight_importance_dict = {}    

    for model_name, model in models:        
        model_hyperparamsearch = GridSearchCV(estimator=model, param_grid=param_grid[model_name], cv=loo, verbose=0, n_jobs=-1)
        model_hyperparamsearch.fit(X_train, y_train)
        best_estimator = model_hyperparamsearch.best_estimator_

        # get prediction on the test dataset
        pred_test = model_hyperparamsearch.predict(X_test)

        # keept test performance results
        result_dict[model_name] = {}
        result_dict[model_name]['uncertainty_mean'] = best_estimator.predict_proba(X_test).max(1).mean()
        result_dict[model_name]['uncertainty_std'] = best_estimator.predict_proba(X_test).max(1).std()
        result_dict[model_name]['test_acc'] = metrics.accuracy_score(y_test, pred_test)
        result_dict[model_name]['test_acc_bal'] = metrics.balanced_accuracy_score(y_test, pred_test)
        result_dict[model_name]['test_f1'] = metrics.f1_score(y_test, pred_test, average='binary')
        result_dict[model_name]['test_cohen_kappa'] = metrics.cohen_kappa_score(y_test, pred_test, labels=y)
        result_dict[model_name]['val'] = model_hyperparamsearch.best_score_
        pprint(result_dict)

        # plot confusion matrix
        cm = metrics.confusion_matrix(y_test, pred_test, labels=y)
        cm = cm/float(cm.sum()) # convert to ratio
        ax = sns.heatmap(cm, annot=True, xticklabels=y, yticklabels=y, cmap='Blues', fmt='.2f', square=True)
        plt.suptitle(f'{model_name}')
        plt.tight_layout()
        plt.savefig(os.path.join(config_path,'reult_dir_3','fig', 'confusion_matrix', model_name+'.png'))
        plt.close()

        # plot ROC curve
        fpr, tpr, thresholds = roc_curve(y_test,pred_test)
        plt.plot(fpr, tpr, 'o-', label="Decision Tree")
        plt.plot([0, 1], [0, 1], 'k--', label="random guess")
        plt.xlabel('Fall-Out')
        plt.ylabel('Recall')
        plt.title('Receiver operating characteristic example')
        plt.savefig(os.path.join(config_path,'reult_dir_3', 'fig', 'roc', model_name+'.png'))
        plt.close()

        # save model and results
        joblib.dump(best_estimator, os.path.join(config_path,'reult_dir_3', 'model', model_name+'.joblib'))
        pd.DataFrame.from_dict(model_hyperparamsearch.cv_results_).to_csv(os.path.join(config_path,'reult_dir_3', 'result', 'cv', model_name+'.csv'))
        pd.DataFrame.from_dict(result_dict).T.to_csv(os.path.join(config_path,'reult_dir_3', 'result', 'result.csv'))
        pd.DataFrame.from_dict(weight_importance_dict).T.to_csv(os.path.join(config_path,'reult_dir_3', 'result', 'importance_weight.csv'))  


In [9]:
main(config_path,target)

Features [ 25  26  27  43  61  62  63  79  97  98  99 115 133 134 135 151 169 170
 171 187 205 206 207 223 241 242 243 259 277 278 279 295 313 314 315 331
 349 350 351 367] are constant.
invalid value encountered in divide


{'Dummy': {'test_acc': 0.5714285714285714,
           'test_acc_bal': 0.5,
           'test_cohen_kappa': 0.0,
           'test_f1': 0.0,
           'uncertainty_mean': 0.5263157894736842,
           'uncertainty_std': 0.0,
           'val': 0.5263157894736842}}



3800 fits failed out of a total of 9120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
760 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\envs\main\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\USER\anaconda3\envs\main\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\Users\USER\anaconda3\envs\main\lib\site-packages\sklearn\linear_model\_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 pe

{'Dummy': {'test_acc': 0.5714285714285714,
           'test_acc_bal': 0.5,
           'test_cohen_kappa': 0.0,
           'test_f1': 0.0,
           'uncertainty_mean': 0.5263157894736842,
           'uncertainty_std': 0.0,
           'val': 0.5263157894736842},
 'LogReg': {'test_acc': 0.7142857142857143,
            'test_acc_bal': 0.7083333333333333,
            'test_cohen_kappa': 0.41666666666666663,
            'test_f1': 0.6666666666666666,
            'uncertainty_mean': 0.9239265084415076,
            'uncertainty_std': 0.05589013696041133,
            'val': 1.0}}
{'Dummy': {'test_acc': 0.5714285714285714,
           'test_acc_bal': 0.5,
           'test_cohen_kappa': 0.0,
           'test_f1': 0.0,
           'uncertainty_mean': 0.5263157894736842,
           'uncertainty_std': 0.0,
           'val': 0.5263157894736842},
 'LogReg': {'test_acc': 0.7142857142857143,
            'test_acc_bal': 0.7083333333333333,
            'test_cohen_kappa': 0.41666666666666663,
            